<a href="https://colab.research.google.com/github/sun-fox/Deep-Learning-CNN/blob/master/Transfer_Learning_Dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 ! pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls ./gdrive/My\ Drive/Colab\ Notebooks/

 Dog_classification.ipynb		       Untitled0.ipynb
 Dog_detector.ipynb			      'Untitled (1).ipynb'
 Handwriiten_digits_recognition_CNN.ipynb      Untitled1.ipynb
 Herbarnium.ipynb			       Untitled2.ipynb
 kaggle.json				       Untitled3.ipynb
 MobileNet_Understanding.ipynb		       Untitled4.ipynb
 Multiclass_classification.ipynb	       Untitled7.ipynb
 Titanic_problem			       Untitled.ipynb
 Traffic_Sign_Recognition.ipynb		       YOLOv3_Tutorial.ipynb
'Transfer Learning Dog_classification.ipynb'


In [0]:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/kaggle.json ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset

 98% 737M/750M [00:08<00:00, 102MB/s]
100% 750M/750M [00:08<00:00, 95.3MB/s]


In [11]:
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/Images/n02108089-boxer/n02108089_117.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_11807.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_11875.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_122.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12232.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_125.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12738.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12739.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12827.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_13340.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_13526.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_1353.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_1355.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_1357.jpg  
  inflating: images/Images/n02108089-boxer/n0210

In [0]:
import os
import numpy as np
import shutil

root_dir = '../content/images/Images/'
classes_dir = [ f.name for f in os.scandir(root_dir) if f.is_dir() ]

val_ratio = 0.15
test_ratio = 0.05

train_root_dir = '../content/images/Images/train/'
test_root_dir = '../content/images/Images/val/'
val_root_dir = '../content/images/Images/test/'

for cls in classes_dir:
    os.makedirs(train_root_dir + cls)
    os.makedirs(val_root_dir + cls)
    os.makedirs(test_root_dir + cls)

    # Creating partitions of the data after shuffeling
    src = root_dir + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - val_ratio + test_ratio)), 
                                                               int(len(allFileNames)* (1 - test_ratio))])

    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, train_root_dir + cls)

    for name in val_FileNames:
        shutil.copy(name, val_root_dir + cls)

    for name in test_FileNames:
        shutil.copy(name, test_root_dir + cls)

# **Transfer Learning Implemented Keras Classifier**

In [0]:
from sklearn.datasets import load_files       
import numpy as np
from glob import glob
from tqdm import tqdm
import cv2
from PIL import ImageFile
import matplotlib.pyplot as plt                        
%matplotlib inline
import keras
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions 
from keras.preprocessing import image

In [0]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../content/images/Images/train/')
valid_files, valid_targets = load_dataset('../content/images/Images/val/')
test_files, test_targets = load_dataset('../content/images/Images/test/')

# load list of dog names
dog_names = [item[41:-1] for item in sorted(glob("../content/images/Images/train/*/"))]

In [15]:
from keras.preprocessing.image import ImageDataGenerator
train_batches= ImageDataGenerator(preprocessing_function = keras.applications.resnet50.preprocess_input).flow_from_directory(train_root_dir,target_size=(224,224),batch_size=100)
val_batches= ImageDataGenerator(preprocessing_function = keras.applications.resnet50.preprocess_input).flow_from_directory(val_root_dir,target_size=(224,224),batch_size=100)
test_batches= ImageDataGenerator(preprocessing_function = keras.applications.resnet50.preprocess_input).flow_from_directory(test_root_dir,target_size=(224,224),batch_size=100,shuffle=False)

Found 18470 images belonging to 120 classes.
Found 1033 images belonging to 120 classes.
Found 1077 images belonging to 120 classes.


#### **Defining ResNet50 Model**

In [0]:
ResNet50_model = ResNet50(weights='imagenet')

102858752/102853048 [==============================] - 4s 0us/step


In [0]:
ResNet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
# Functions that process images before sending to CNN
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)
    
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [0]:
def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

In [0]:
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151))

In [0]:
# # Model Architecture
# model1 = Sequential()
# model1.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
# model1.add(MaxPooling2D(pool_size=2))
# 1
# model1.add(Conv2D(filters=32, kernel_size=2 , padding='same' , activation='relu'))
# model1.add(MaxPooling2D(pool_size=2))
# model1.add(Conv2D(filters=64 , kernel_size=2 , padding='same' , activation='relu'))
# model1.add(MaxPooling2D(pool_size=2))
# model1.add(Dropout(0.4))
# model1.add(Conv2D(filters=128 , kernel_size=2 , padding='same' , activation='relu'))
# model1.add(MaxPooling2D(pool_size=2))
# model1.add(Dropout(0.4))
# model1.add(Flatten())
# model1.add(Dense(512, activation='relu'))
# model1.add(Dropout(0.4))
# model1.add(Dense(120,activation='softmax'))
# model1.summary()

In [0]:
# sgd = SGD(lr= 1e-3 , decay=1e-6, momentum=0.9 , nesterov = True)
# model1.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
# model1.fit(x_train, y_train, epochs=100)

In [0]:
# loss, accuracy = model1.evaluate(x_test, y_test)

# print(loss,accuracy)

In [0]:
# pred = model1.predict(x_test)

# pred.shape

**Applying transfer learning to the ResNet50 model**

In [0]:
from keras.models import Model
x = ResNet50_model.layers[-4].output
y = Flatten()(x)
predictions = Dense(120,activation='softmax')(y)
model = Model(inputs=ResNet50_model.input, outputs = predictions)

In [0]:
for layer in model.layers[:-3]:
  layer.trainable = False

In [0]:
from keras.optimizers import Adam
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
model.fit_generator(train_batches,steps_per_epoch=185,validation_data=val_batches,validation_steps=10 ,epochs=15,verbose=2)

Epoch 1/15
 - 193s - loss: 6.2250 - accuracy: 0.4357 - val_loss: 4.2895 - val_accuracy: 0.6170
Epoch 2/15
 - 178s - loss: 1.3001 - accuracy: 0.8099 - val_loss: 3.8459 - val_accuracy: 0.6141
Epoch 3/15
 - 177s - loss: 0.7431 - accuracy: 0.8809 - val_loss: 3.7922 - val_accuracy: 0.6292
Epoch 4/15
 - 177s - loss: 0.5400 - accuracy: 0.9168 - val_loss: 4.1705 - val_accuracy: 0.6463
Epoch 5/15
 - 177s - loss: 0.4975 - accuracy: 0.9227 - val_loss: 4.5339 - val_accuracy: 0.6613
Epoch 6/15
 - 177s - loss: 0.4643 - accuracy: 0.9359 - val_loss: 4.9589 - val_accuracy: 0.6527
Epoch 7/15
 - 177s - loss: 0.4307 - accuracy: 0.9428 - val_loss: 7.4782 - val_accuracy: 0.6302
Epoch 8/15
 - 177s - loss: 0.5342 - accuracy: 0.9366 - val_loss: 4.9420 - val_accuracy: 0.6731
Epoch 9/15
 - 177s - loss: 0.4362 - accuracy: 0.9461 - val_loss: 4.1310 - val_accuracy: 0.6506
Epoch 10/15
 - 177s - loss: 0.4755 - accuracy: 0.9466 - val_loss: 6.3458 - val_accuracy: 0.6785
Epoch 11/15
 - 177s - loss: 0.4050 - accuracy: 0.

# **Transfer Learning on MobileNet Model**

In [0]:
mobile = keras.applications.mobilenet.MobileNet()

In [48]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [0]:
from keras.models import Model
x = mobile.layers[-6].output
# y = Flatten()(x)
predictions = Dense(120,activation='softmax')(x)
model = Model(inputs=mobile.input, outputs = predictions)

In [0]:
for layer in model.layers[:-5]:
  layer.trainable = False

In [0]:
from keras.optimizers import Adam
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [52]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [53]:
model.fit_generator(train_batches,steps_per_epoch=185,validation_data=val_batches,validation_steps=10 ,epochs=10,verbose=2)

Epoch 1/10
 - 79s - loss: 3.6672 - accuracy: 0.2035 - val_loss: 4.6278 - val_accuracy: 0.0520
Epoch 2/10
 - 78s - loss: 2.0527 - accuracy: 0.5281 - val_loss: 4.2260 - val_accuracy: 0.0750
Epoch 3/10
 - 77s - loss: 1.4997 - accuracy: 0.6552 - val_loss: 4.3833 - val_accuracy: 0.0782
Epoch 4/10
 - 77s - loss: 1.1996 - accuracy: 0.7308 - val_loss: 4.1893 - val_accuracy: 0.0793
Epoch 5/10
 - 77s - loss: 1.0045 - accuracy: 0.7799 - val_loss: 4.4616 - val_accuracy: 0.0825
Epoch 6/10
 - 77s - loss: 0.8633 - accuracy: 0.8215 - val_loss: 4.3742 - val_accuracy: 0.0665
Epoch 7/10
 - 77s - loss: 0.7487 - accuracy: 0.8523 - val_loss: 4.4679 - val_accuracy: 0.0772
Epoch 8/10
 - 77s - loss: 0.6512 - accuracy: 0.8820 - val_loss: 4.1364 - val_accuracy: 0.0932
Epoch 9/10
 - 77s - loss: 0.5693 - accuracy: 0.9048 - val_loss: 4.4985 - val_accuracy: 0.0718
Epoch 10/10
 - 78s - loss: 0.4996 - accuracy: 0.9277 - val_loss: 4.7249 - val_accuracy: 0.0654


In [0]:
test_labels = test_batches.classes

In [55]:
test_labels

array([  0,   0,   0, ..., 119, 119, 119], dtype=int32)

In [68]:
dictionary = test_batches.class_indices
type(dictionary)

dict

In [74]:
predictions = model.predict_generator(test_batches,steps=100,verbose=1)
predictions = predictions.tolist()

100/100 [==============================] - 44s 436ms/step


In [77]:
for idx,cls in enumerate(predictions):
  max=0;
  maxpos=0;
  for i in range(119):
    if cls[i]>max:
      max = cls[i]
      maxpos = i
  print(list(dictionary.keys())[list(dictionary.values()).index(maxpos)][10:] + " for " +list(dictionary.keys())[list(dictionary.values()).index(idx)][10:])

Lakeland_terrier for Chihuahua
Norwich_terrier for Japanese_spaniel
Chihuahua for Maltese_dog
toy_terrier for Pekinese
Yorkshire_terrier for Shih-Tzu
Airedale for Blenheim_spaniel
Brittany_spaniel for papillon
Pembroke for toy_terrier
Airedale for Rhodesian_ridgeback
Great_Pyrenees for Afghan_hound
wire-haired_fox_terrier for basset
Japanese_spaniel for beagle
Yorkshire_terrier for bloodhound
Japanese_spaniel for bluetick
Old_English_sheepdog for black-and-tan_coonhound
Japanese_spaniel for Walker_hound
Lakeland_terrier for English_foxhound
Welsh_springer_spaniel for redbone
Maltese_dog for borzoi
toy_poodle for Irish_wolfhound
Old_English_sheepdog for Italian_greyhound
Maltese_dog for whippet
silky_terrier for Ibizan_hound
Lakeland_terrier for Norwegian_elkhound
Old_English_sheepdog for otterhound
silky_terrier for Saluki
Maltese_dog for Scottish_deerhound
Maltese_dog for Weimaraner
Old_English_sheepdog for Staffordshire_bullterrier
Maltese_dog for American_Staffordshire_terrier
Lakel

ValueError: ignored